## Custom Linear Regression Function

-Varishu Pant

#### Aim-
This function aims to fit linear regression model on given data set while giving user the ability to choose between solution approach,and validation techniques.

### Importing Required Libraries

In [1]:
import pandas as pd
import numpy as np
import statsmodels.tools as stm
import statsmodels.api as stmapi
import time

import matplotlib.pyplot as plt

#### MtCars Dataset Used for Demonstration

In [24]:
data=pd.read_csv(r'C:\Users\Varishu Pant\Desktop\Praxis docs\Excel\cars.csv')

In [94]:
data.shape

(406, 9)

In [97]:
data.describe()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model
count,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000
mean,23.051232,5.475369,194.779557,103.529557,2979.413793,15.519704,75.921182
std,8.401777,1.712160,104.922458,40.520659,847.004328,2.803359,3.748737
min,0.000000,3.000000,68.000000,0.000000,1613.000000,8.000000,70.000000
25%,17.000000,4.000000,105.000000,75.000000,2226.500000,13.700000,73.000000
50%,22.350000,4.000000,151.000000,93.500000,2822.500000,15.500000,76.000000
75%,29.000000,8.000000,302.000000,129.000000,3618.250000,17.175000,79.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000


For Open Form Solution,function normalizes data

### Custom Linear Regression Function

Takes in dataframe,and list of variables for predictors(x) and target(y)
User can choose between open or closed form solution and 2 cross validation techniques-namely K-Fold Cross validation and Leave one out cross validation

In [117]:
def custom_lr(data,x,y):
    soln=input('Solution Form:Open[Gradient Descent](O) or Closed[OLS](C):')
    validation=input('Cross-Validation:K-Fold(k) or Leave One out(l):')
    if validation=='k':
        ks=int(input('Enter k:'))
        cv_predict(data,x,y,soln,k=ks)
    elif validation=='l':
        loocv_predict(data,x,y,soln)
    else:
        print('Invalid Entry,type k or l')

### Demonstration

#### Open Form using 5-Fold C.V

In [93]:
custom_lr(data,x=['Weight','Horsepower','Cylinders'],y='MPG')

Solution Form:Open[Gradient Descent](O) or Closed[OLS](C):O
Cross-Validation:K-Fold(k) or Leave One out(l):k
Enter k:5
Learning Rate:0.001
Tolerance:0.01
One of the models for your general understanding:
Intercept: 0.7802936012936587
Parameters: {'B1': -0.4072746955342833, 'B2': -0.21658122749392017, 'B3': -0.05763295259645001}
Average RSS: 354.9169613882736


Function asks for open or closed form solution-Input given was O for Open form
Then function asks for cross validation techniques-Input given was k for K-Fold.
Function asks for number of folds(K)- input given was 5 (Default =5)
Since Open form was selected,function asks for tolerance and learning rate..
Out of all models trained,function spits out last one for a general understanding of the weights.
Also average Residual Sum of Squares is printed for model accuracy

#### Open Form using Leave One out C.V

In [98]:
custom_lr(data,x=['Weight','Horsepower','Acceleration'],y='MPG')

Solution Form:Open[Gradient Descent](O) or Closed[OLS](C):O
Cross-Validation:K-Fold(k) or Leave One out(l):l
Learning Rate:0.001
Tolerance:0.01
One of the models for your general understanding:
Intercept: 0.742565642343966
Parameters: {'B1': -0.46658815229820444, 'B2': -0.1846693392596126, 'B3': 0.035078957700797005}
Average RSS: 8.948412310636611


#### Closed Form using 4-Fold C.V

In [103]:
normalized_data=data[['Weight','Horsepower','Displacement','Cylinders','MPG']].apply(lambda x:(x-np.min(x))/(np.max(x)-np.min(x)))

custom_lr(normalized_data,x=['Weight','Horsepower','Cylinders'],y='MPG')  

Solution Form:Open[Gradient Descent](O) or Closed[OLS](C):C
Cross-Validation:K-Fold(k) or Leave One out(l):k
Enter k:4
One of the models for your general understanding:
Weights: [[ 0.75143278]
 [-0.42222072]
 [-0.11877358]
 [-0.07514557]]

Average RSS: 1.5938611613518017


#### Closed Form using LOOCV

To get similar results using closed form solution,we feed normalized data to the function

In [104]:
normalized_data=data[['Weight','Horsepower','Displacement','Cylinders','MPG']].apply(lambda x:(x-np.min(x))/(np.max(x)-np.min(x)))

custom_lr(normalized_data,x=['Weight','Horsepower','Displacement','Cylinders'],y='MPG')  

Solution Form:Open[Gradient Descent](O) or Closed[OLS](C):C
Cross-Validation:K-Fold(k) or Leave One out(l):l
One of the models for your general understanding:
Weights: [[ 0.7627041 ]
 [-0.40238162]
 [-0.19694794]
 [-0.02759752]
 [-0.02975016]]
Average RSS: 5.235262530734401


### Functions used behind the scenes-

#### K-Fold Cross Validation

In [112]:
def cv_predict(data,x,y,soln,k=5):
    import math
    import numpy as np
    if k==0 or k==1 or k<0:
        return('Value Error:Try k>1')
    
    cv_error=[]
    if soln=='C':
        hold_indices=[]
        
        fold_size=1/k
        for i in range(k):
            data_indices=[]
            for i in data.index:
                if i not in hold_indices:
                    data_indices.append(i)
            hold_indices=np.random.choice(data_indices, round(fold_size*len(data.index)), replace=False)
            train_indices=[]
            for j in data.index:
                if j not in hold_indices:
                    train_indices.append(j)
            hold_data=data.loc[hold_indices,:]
            train_data=data.loc[train_indices,:]
            X_train=train_data[x]
            Y_train=train_data[[y]]
            X_test=hold_data[x]
            Y_test=hold_data[[y]]

        
            b=mlrm(x,y,xtrain=X_train,ytrain=Y_train)
            preds=[]
            for j in range(0,X_test.shape[0]):
                xrow=X_test[j:j+1].values[0]
                test=0
            
                for i in range(1,len(b)):
                    c=0
                    test+=b[i]*xrow[c]
                    c+=1
                pred=test+b[0]
                preds.append(pred)
            e = np.array(Y_test) - np.array(preds)
            rss= np.sum(e**2)
            
            cv_error.append(rss)
        print('One of the models for your general understanding:')
        print('Weights:',b)
        
        print()
        avg_error=np.mean(cv_error)
  
    
    elif soln=='O':
        learning_rate=float(input('Learning Rate:'))
        tolerance=float(input('Tolerance:'))   
        hold_indices=[]
        subset=x.copy()
        subset.append(y)
        data=data[subset].apply(lambda x:(x-np.min(x))/(np.max(x)-np.min(x)))

        fold_size=1/k
        for i in range(k):
            data_indices=[]
            for i in data.index:
                if i not in hold_indices:
                    data_indices.append(i)
            hold_indices=np.random.choice(data_indices, round(fold_size*len(data.index)), replace=False)
            train_indices=[]
            for j in data.index:
                if j not in hold_indices:
                    train_indices.append(j)
            hold_data=data.loc[hold_indices,:]
            train_data=data.loc[train_indices,:]
            X_train=train_data[x]
            Y_train=train_data[[y]]
            X_test=hold_data[x]
            Y_test=hold_data[[y]]

            dict = lr_with_gd(xtrain=X_train,ytrain=Y_train,y=y,x=x,tolerance=tolerance,learning_rate=learning_rate)

            intercept=dict.pop('B0')
            preds=[]
            for j in range(0,X_test.shape[0]):
                xrow=X_test[j:j+1].values[0]
                test=0
                for i in range(1,len(dict.keys())):
                    c=0
                    test+=dict['B'+str(i)]*xrow[c]
                    c+=1
                pred=test+intercept
                preds.append(pred)
            e = np.array(Y_test) - np.array(preds)
            rss=np.sum(e**2)
            cv_error.append(rss)
        avg_error=np.mean(cv_error)
        print('One of the models for your general understanding:')
        print('Intercept:',intercept)
        print('Parameters:',dict)
    else:
        print('Invalid Entry,Type O or C')
        return()
    print('Average RSS:',avg_error)
    

#### Leave One Out Cross Validation

In [113]:
def loocv_predict(data,x,y,soln):
    loocv_error=[]
    if soln=='O':
        learning_rate=float(input('Learning Rate:'))
        tolerance=float(input('Tolerance:'))
        subset=x.copy()
        subset.append(y)

        data=data[subset].apply(lambda x:(x-np.min(x))/(np.max(x)-np.min(x)))
        rss=0
        for i in range(0,len(data.index)):
            hold_data=data.loc[i,:]
            
            X_test=hold_data[x]
            Y_test=hold_data[[y]]
            train_data=data.drop(index=i,axis=0)
            X_train=train_data[x]
            Y_train=train_data[[y]]

            dict = lr_with_gd(xtrain=X_train,ytrain=Y_train,y=y,x=x,tolerance=tolerance,learning_rate=learning_rate)
            test=0
            
            intercept=dict.pop('B0')
            for i in range(1,len(dict.keys())):
                test+=dict['B'+str(i)]*X_test[i]
                
            pred=test+intercept
            e = Y_test - pred
            rss +=e**2
            loocv_error.append(rss)
        print('One of the models for your general understanding:')
        print('Intercept:',intercept)
        print('Parameters:',dict)
            
    elif soln=='C':
        rss=0
        for i in range(0,len(data.index)):
            hold_data=data.loc[i,:]
            X_test=hold_data[x]
            Y_test=hold_data[[y]]
            train_data=data.drop(index=i,axis=0)
            X_train=train_data[x]
            Y_train=train_data[[y]]

            b=mlrm(x,y,xtrain=X_train,ytrain=Y_train)
            test=0
 
            for i in range(1,len(b)):
                c=0
                test+=b[i]*X_test[c]
                c+=1
            pred=test+b[0]
            e = Y_test - pred
            rss +=e**2 
            loocv_error.append(rss)
           
        print('One of the models for your general understanding:')
        print('Weights:',b)
    else:
        print('Invalid entry,Type O or C')
        return()
    avg_error=np.mean(loocv_error)
    print('Average RSS:',avg_error)

#### Open Form Solution (Gradient Descent)

In [5]:
def lr_with_gd(xtrain,ytrain,x,y,tolerance,learning_rate):
        xs=len(x)
        y=ytrain
        x=xtrain
        Y=np.matrix(y)                                                  
        Y=Y.reshape(Y.shape[0],1)

        X=stm.add_constant(x)                                          #Adding column for calculating intercept
        X=np.matrix(X)

        b=np.array(np.zeros(X.shape[1]))                               #Initializing weights
        b=b.reshape(b.shape[0],1)
        dict={}
        
        for i in range(xs+1):
            dict['B'+str(i)]=0
        RSS=[] 
        niter=0
        convergence_criteria=1                                         #Initializing convergence criteria    
        iter=[]

          
        while convergence_criteria > tolerance:                        #Loop runs til convergence criteria is greater
                                                                       #than tolerance level
            niter+=1                                                   #Counting iterations
            iter.append(niter)

            rss=np.matmul(np.transpose(X),(Y - np.matmul(X, b)))       #Calculating Residual Sum of Squares

            convergence_criteria=np.abs(np.linalg.norm(rss))           #Calculating convergence criteria
            RSS.append(convergence_criteria)

            b=b+2*learning_rate*rss                                    #Updating weights

    
        for i in range(xs+1):
            dict['B'+str(i)]=float(b[i])
        return(dict)

#### Closed Form Solution(OLS)

In [20]:
def mlrm(x,y,xtrain,ytrain):
    x=xtrain[x]
    y=ytrain[[y]]
    x=stm.add_constant(x.values)
    xt=np.transpose(x)
    b=np.matmul(np.matmul(np.linalg.inv(np.matmul(xt,x)),xt),y.values)
    return(b)
